In [1]:
import os
import pandas as pd
import numpy as np
import math
from multiprocessing import Pool
from itertools import combinations
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from statsmodels.tsa.statespace.sarimax import SARIMAX

import warnings
warnings.filterwarnings('ignore')
def log_transf(x):
  if (x != 0 and x != np.nan):
    return np.log(x)
  if (x == 0 and x != np.nan):
    return 0
  if x == np.nan:
    return np.nan

C:\Users\subas\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\subas\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\subas\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def predict_new_deaths(window,train,p,q,ps,qs,w,d,ds):
  col_core = 'new_deaths'
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])
  df_US_log = pd.DataFrame()
  for col in df_US.columns:
    df_US_log[col] = df_US[col].apply(lambda x: log_transf(x))

  df_nd1 = df_US_log[col_core]
  df_hp = df_US_log['hosp_patients']
  df_ip = df_US_log['icu_patients']
  df_nc = df_US_log['new_cases']
  df_nt = df_US_log['new_tests']

  for h in range(1,5):
      x_lagged_hosp = df_hp.shift(h).bfill()
      x_lagged_icu = df_ip.shift(h).bfill()
      x_lagged_nc = df_nc.shift(h).bfill()
      x_lagged_nt = df_nt.shift(h).bfill()

      exog_variables = pd.DataFrame()
      exog_variables1 = pd.concat([x_lagged_icu, x_lagged_hosp], axis=1)
      exog_variables2 = pd.concat([exog_variables1, x_lagged_nc], axis=1)
      exog_variables3 = pd.concat([exog_variables2, x_lagged_nt], axis=1)

      df_current_old14 = df_nd1[int(window-1):int(window-1+train+h)]

      i = h-1
      col_train = df_nd1.loc[df_current_old14.index][0:train]


      exo_train = exog_variables2.loc[df_current_old14.index][0:train]
      exo_test = exog_variables2.loc[df_current_old14.index][train:]


      model = SARIMAX(col_train, exog=exo_train, order=(h,d,q), seasonal_order=(ps,ds,qs,w), enforce_stationarity=False, enforce_invertibility=False)
      model_fit = model.fit(disp=False)
      forecast = model_fit.predict(len(col_train), len(col_train)+i, exog = exo_test)
      forecast = np.exp(forecast)
      forecast_f = forecast.to_frame()

      if h == 1:
        df1 = forecast_f.rename(columns= {0: 'new_deaths'})
        value1 = df1['new_deaths'].iloc[i]
        index1 = df1['new_deaths'].index[i]


      elif h == 2:
        df2 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value2 = df2['new_deaths'].iloc[i]
        index2 = df2['new_deaths'].index[i]


      elif h == 3:
        df3 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value3 = df3['new_deaths'].iloc[i]
        index3 = df3['new_deaths'].index[i]


      elif h == 4:
        df4 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value4 = df4['new_deaths'].iloc[i]
        index4 = df4['new_deaths'].index[i]


        data = {'index': [index1, index2, index3, index4],'new_deaths': [value1, value2, value3, value4]}
        forecast_f = pd.DataFrame(data=data)
        forecast_f = forecast_f.set_index('index')


  return forecast_f

In [3]:
def window_results_nd(n_train, n_test,p,q,ps,qs,w,d,ds):

  col_core = 'new_deaths'
  temp_results = pd.DataFrame()
  for i in range(n_test):
    window = i + 1
    predict_current = predict_new_deaths(window,n_train,p,q,ps,qs,w,d,ds)
    predict_current = predict_current.rename(columns={str(col_core):"window_"+str(window)})
    temp_results = pd.concat([temp_results, predict_current], axis=1)

  return temp_results

In [4]:
def output_h_nd(temp_results, h, n_test):
  n_iter = n_test-h+1
  df_results = []
  if h==1:
    for i in range(int(n_iter)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
  if h>1:
    for i in range(int(h-1)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())

    for i in range(int(n_iter)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())
  df_results_python = pd.concat(df_results,axis=0)
  return df_results_python

In [5]:
def predict_table_nd(n_test, n_train ,p,q,ps,qs,w,d,ds,temp_results):
  df_final_forecast = pd.DataFrame()
  for h in range(1,5):
    # temp_results = window_results_nd(n_train, n_test,p,q,ps,qs,h,w,d,ds)
    df_current = output_h_nd(temp_results, h, n_test)
    df_final_forecast = pd.concat([df_final_forecast, df_current], axis=1)
  return df_final_forecast

In [6]:
def smape_results_nd(df_final_forecast, p):
  col_core = 'new_deaths'
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  # df_predict = np.exp(df_final_forecast)
  h_level = df_final_forecast.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_final_forecast.index][col_core]
    y_pred = df_final_forecast[cols]
#     print(y_true,y_pred)
    value = smape(y_true, y_pred)
    eval.append(value)
  eval.append("{:.2f}".format(np.mean(eval)))
  df_results = pd.DataFrame(eval, columns=[str(col_core)+"_p="+str(p)])
  h_level.append('Average')
  df_results.index = h_level
  return df_results

In [7]:
def smape(y_true, y_pred):
  smape = (+1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100))
  return round(smape,2)

In [8]:
def mae_results(df_final_forecast, p):
  col_core = 'new_deaths'
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  # df_predict = np.exp(df_final_forecast)
  h_level = df_final_forecast.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_final_forecast.index][col_core]
    y_pred = df_final_forecast[cols]
    value = mae(y_true, y_pred)
    eval.append(value)
  eval.append("{:.2f}".format(np.mean(eval)))
  df_results = pd.DataFrame(eval, columns=[str(col_core)+"_p="+str(p)])
  h_level.append('Average')
  df_results.index = h_level

  return df_results

In [9]:
def mae(y_true, y_pred):
  error = 0
  for i in range(len(y_true)):
        error += abs(y_true[i] - y_pred[i])
  mae = (error / len(y_true))
  return mae

In [10]:
def relative_absolute_error(true, pred):
    true_mean = np.mean(true)
    squared_error_num = np.sum(np.abs(true - pred))
    squared_error_den = np.sum(np.abs(true - true_mean))
    rae_loss = squared_error_num / squared_error_den
    return rae_loss

In [12]:
#For reproducibility cdc results
import warnings
warnings.filterwarnings('ignore')
n_train = 40
n_test = 70
y = 'new_deaths'
m = 10
p = 1
ps = 1
qs = 0
d = 1
q = 1
ds = 0
h_max = 4
x = ['icu_patients','positive_rate']
print(p,d,q,ps,ds,qs)
temp_results = window_results_nd(n_train, n_test,p,q,ps,qs,w,d,ds)
df_final_forecast = predict_table_nd(n_test, n_train, p,q, ps, qs,w,d,ds,temp_results)
print(df_final_forecast)
df_smape_results = smape_results_nd(df_final_forecast, p)
print("sMAPE {}".format(df_smape_results))
df_mae_results = mae_results(df_final_forecast,p)
print("MAE {}".format(df_mae_results))
# df_mrae_results = mrae_results_nd(df_final_forecast)
# print("MRAE {}".format(df_mrae_results))
print(p,d,q,ps,ds,qs,w)

KeyboardInterrupt: 

In [11]:
def mrae_results_nd(df_final_forecast,df_US_CDC,p, model):
  col_core = 'new_deaths'
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  # df_predict = np.exp(df_final_forecast)
  h_level = df_US_CDC.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_US_CDC.index][col_core]
    y_pred = df_US_CDC[cols]
    # if cols == 'h=1':
    #     plt.figure(figsize=(10, 6))
    #     plt.plot(df_US[col_core], label='True')
    #     plt.plot(df_final_forecast[cols], label='Predicted_SARIMAX', color='purple')
    #     if model is not None:
    #         plt.plot(df_US_CDC[cols], label='Predicted_'+model)
    #         plt.title(model)
    #     else:
    #         plt.title('SARIMAX Prediction vs. Test Data')
    #     plt.xlabel('Date')
    #     plt.ylabel('New Deaths')
    #     plt.legend()
#         plt.show()
    #print(y_true,y_pred)
    value = relative_absolute_error(y_true, y_pred)
    eval.append(value)
  eval.append("{:.2f}".format(np.mean(eval)))
  df_results = pd.DataFrame(eval, columns=[str(col_core)+"_p="+str(p)])
  h_level.append('Average')
  df_results.index = h_level
  return df_results

In [22]:
def cdc_data(model):
    df_US_CDC = pd.read_csv('concatenated_CDC_20_21_22_23.csv', on_bad_lines='skip')
    df_US_CDC = df_US_CDC[(df_US_CDC['Model'] == model)]
    df_US_CDC = df_US_CDC[['Date Model was run', 'Point']]
    df_US_CDC['Date Model was run'] = pd.to_datetime(df_US_CDC['Date Model was run'])
    df_US_CDC = df_US_CDC.sort_values('Date Model was run').groupby('Date Model was run')['Point'].apply(lambda df: df.reset_index(drop=True)).unstack().reset_index()
    # print(df_US_CDC)
    df_US_CDC.set_index('Date Model was run', inplace=True)
    df_US_CDC.columns =['h=1', 'h=2', 'h=3', 'h=4']
    pd.set_option('display.max_rows', 10)
    start_date = pd.to_datetime('2020-12-05')
    end_date = pd.to_datetime('2022-04-02')
    df_US_CDC_1 = df_US_CDC.loc[start_date:end_date]
    print(model)
    print(df_US_CDC_1)
    df_smape_results = smape_results_nd(df_US_CDC_1,1)
    df_smape_results.columns = [''] * len(df_smape_results.columns)
    print("{}".format(df_smape_results))
    # df_mae_results = mae_results(df_US_CDC,1)
    # df_mae_results.columns = [''] * len(df_mae_results.columns)
    # print("{}".format(df_mae_results.tail(1)))
    # df_mrae_results = mrae_results_nd(df_final_forecast,df_US_CDC,1,model)
    # df_mrae_results.columns = [''] * len(df_mrae_results.columns)
    # print("{}".format(df_mrae_results.tail(1)))
    print("####################")

In [23]:
cdc_data("BPagano")
cdc_data("Columbia")
cdc_data("MIT-LCP")
cdc_data("CovidComplete")
cdc_data("ESG")
cdc_data("GT-DeepCOVID")
cdc_data("JHU-APL")
cdc_data("Karlen")
cdc_data("MIT-ORC")
cdc_data("MOBS")
cdc_data("PSI")
cdc_data("UCSD-NEU")
cdc_data("UM")

BPagano
                      h=1    h=2    h=3    h=4
Date Model was run                            
2020-12-05          19457  21096  21552  17009
2020-12-12          20225  22936  24424  24493
2020-12-19          21893  22530  20106  21155
2020-12-26          20039  20940  21305  18684
2021-01-02          20020  20755  20675  20272
...                   ...    ...    ...    ...
2022-03-05           7353   5931   4651   3511
2022-03-12           6319   4902   3774   2868
2022-03-19           1768   2413   3284   4443
2022-03-26           3750   3064   2464   4493
2022-04-02           3664   3250   2871   2530

[69 rows x 4 columns]
              
h=1       25.0
h=2      27.01
h=3      29.25
h=4      26.24
Average  26.88
####################
Columbia
                      h=1    h=2    h=3    h=4
Date Model was run                            
2020-12-05          13457  14441  15101  12537
2020-12-12          20683  17666  19190  20387
2020-12-19          18922  19444  18945  19138
202

In [27]:
# cdc_data("Columbia")
# cdc_data("MIT-LCP")
# cdc_data("BPagano")
# cdc_data("Covid19Sim")
# # cdc_data("CovidComplete")
# # cdc_data("ESG")
# # cdc_data("GT-DeepCOVID")
# cdc_data("IHME")
# # cdc_data("JHU-APL")
# cdc_data("JHU-IDD")
# # cdc_data("Karlen")
# cdc_data("Microsoft")
# cdc_data("MIT-ISOLAT")
# # cdc_data("MIT-ORC")
# # cdc_data("MOBS")
# # cdc_data("PSI")
# cdc_data("QJHong")
# cdc_data("UCSD-NEU")
# cdc_data("UM")
# cdc_data("UMass-MB")

# # cdc_data("Alpert")
# cdc_data("Columbia-UNC")
# cdc_data("DDS")
# cdc_data("IEM")
# cdc_data("ISU")
cdc_data("JCB")
cdc_data("JHU-CSSE")
cdc_data("LANL")
# cdc_data("LUcompUncertLab")
cdc_data("Masaryk")
cdc_data("NotreDame-Mobility")
cdc_data("Oliver-Wyman")
cdc_data("RPI-UW")
cdc_data("UA")
cdc_data("UCLA")
cdc_data("UCM")
cdc_data("UGA-CEID")
cdc_data("UpstateSU")
cdc_data("UT")

JCB
                      h=1    h=2    h=3    h=4
Date Model was run                            
2020-12-05          17071  19031  21201  15212
2020-12-19          20147  22275  24698  27473
2020-12-26          22937  19858  21309  24792
              
h=1      14.86
h=2      16.97
h=3       26.2
h=4       23.8
Average  20.46
####################
JHU-CSSE
                      h=1    h=2    h=3    h=4
Date Model was run                            
2020-12-05          16716  16939  15952  14779
2020-12-12          20799  22400  23448  18468
2020-12-19          19321  20298  20991  21364
2020-12-26          18634  18467  17968  17413
2021-01-23          21324  21919  22055  22476
...                   ...    ...    ...    ...
2021-12-25           9218   9325  10207  10658
2022-01-01          15536  13615  11339  10291
2022-02-19          12545  10456   9334   8410
2022-02-26          10190   8548   7396   6225
2022-03-05           6783   9019   7753   5895

[53 rows x 4 columns]
       

In [ ]:
  df_US = pd.read_csv('OWID_weekly.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])